# The Easy "Hard" Way: Cythonizing

[Cython](http://cython.org/) is a **compiler** and a **programming language** used to generate C extensions for Python.

The Cython language is a Python/C [creole](https://en.wikipedia.org/wiki/Creole_language) which is essentially Python with some additional keywords for specifying static data types. It looks something like this:

```cython
def cython_sum(int n):
    cdef float s = 0.0
    cdef int i
    for i in range(n):
        s += i
    return s
```

You can write in this language, then use the Cython compiler to *transpile* to efficient C code which can in turn be compiled into a Python extension module. This extension module can be imported like a normal Python module, but it is backed by compiled C, offering potentially drastic speedups over equivalent functions written in pure Python.

The other major use case for Cython -- the one we will focus on here -- is writing wrappers around existing C code so that the functions therein can be made available in an extension module as described above. We will use this technique to make the SymPy-generated C code accessible to Python for use in SciPy's `odeint`.

**Learning Objectives**

After this lesson, you will be able to:

- write a simple Cython function and run it a Jupyter notebook using the `%%cython` magic command
- use Cython to wrap a function implemented in plain C and make it callable from Python
- use the SymPy `codegen` function to output compilable C code
- wrap `codegen`-generated code with Cython, compile it into an extension module, and call it from Python
- use SymPy's `autowrap` function to do all of this behind the scenes
- pass a custom code printer to `autowrap` to make use of an efficient C library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sm
sm.init_printing()

## 1. Introduction to Cython

Cython is used in two scenarios:

1. writing Cython computation code (i.e. Python with statically typed variables) to generate fast extension modules
2. writing Cython wrappers around existing C code

In both cases, efficient C code can be generated and made available via import from Python.

### Writing Cython Code

In the first use case, you write in the Cython language, compile this into CPython C code, then compile it into a Python extension module. There are several ways to ways to go about the compilation process, and in many cases, Cython's tooling makes it fairly simple. For example, Jupyter notebooks can make use of a `%%cython` magic command that will do all of compilation in the background for us. To make use of it, we need to load the `cython` extension.

In [ ]:
%load_ext cython

Now we can write a Cython function. Note that the `--annotate` (or `-a`) flag of the `%%cython` magic command will produce an interactive annotated printout of the Cython code, allowing us to see the C code that is generated.

In [ ]:
%%cython --annotate
def cython_fib(int n):
    cdef int i
    cdef double a = 0.0, b = 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%timeit cython_fib(100)

**Exercise 1**: remove the type declarations to turn the Cython code above into a plain Python function and use `%timeit` to compare performance.

```python
def python_fib(n):
    # fill in implementation
```

In [ ]:
def python_fib(n):
    a = 0.0
    b = 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%timeit python_fib(100)

To see a bit more about writing Cython and its potential performance benefits, see [this Cython examples notebook](cython-examples.ipynb).

## 2. Wrapping C Code using Cython

Our main goal in using Cython is to make numerical code generated by SymPy available to Python. To achieve this, we'll focus on the second use case for Cython -- wrapping existing C code. Wrapping a set of C functions involves wrapping existing C code with a fairly simple Cython script that specifies the Python interface to the C functions. This script must do two things:

1. specify the function signatures as found in the C header file
2. implement the Python interface to the functions by wrapping them

### Example

In this example, we'll walk through wrapping a simple C library that generates an array of the values in the [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_number):

$$
F_i = F_{i-1} + F_{i-2}
$$

initialized with $F_0 = 0$ and $F_1 = 1$. Here is the C code that does this.

In [ ]:
%cat ../scipy2017codegen/cfib/cfib.c

A function is provided in our tutorial package to simplify the build configuration so that our `cfib.{c,h}` files are found, built, and linked correctly with our wrapper. It takes the name of the module we're creating, then some keyword arguments:

- `sources`: C source files that need to be built
- `include_dirs`: locations of header files. In our case, both the `cfib.h` file and the NumPy headers need to be available.

In [ ]:
# get a path to the cfib source
import os
import scipy2017codegen
cfib_path = os.path.join(os.path.dirname(scipy2017codegen.__file__), 'cfib')

# generate a "pyxbld" file that specifies how to build everything
from scipy2017codegen.templates import render_pyxbld
render_pyxbld('fib',
              sources=[os.path.join(cfib_path, 'cfib.c')],
              include_dirs=[np.get_include(), cfib_path])

Now we can write our wrapper code.

We make use of the `%%cython_pyximport` magic function, which is able to read the Cython code in the cell, compile it using our `pyxbld` file, then import its functions as though we had just written a Python function in a normal code cell.

To write the wrapper, you first write the function signature as specified by the C library. Then, you create a *wrapper function* that makes use of the implementation and returns the result. This wrapper function becomes the interface to the compiled code.

[ndarray.data](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.data.html#numpy.ndarray.data)

In [ ]:
%%cython_pyximport fib
import numpy as np
cimport numpy as np

# here we just replicate the function signature from the header
cdef extern from "cfib.h":
    void cfib(int n, double *x)
    
# here is the "wrapper"
def fib(int n):
    # preallocate an array of doubles
    cdef np.ndarray[np.float64_t, ndim=1] x = np.empty(n, dtype=np.float64)
    
    # call the c function and return the result
    cfib(n, <double *> x.data)
    return x

Now we can call the function `fib` to check that everything worked.

In [ ]:
fib(10)

### Using `codegen()` to Generate C Code

In the previous notebook focusing on generating C code using SymPy's codegen capabilities, we saw how to form a Jacobian from a system of ODEs, then output C code that can numerically evaluate the right hand side of the ODEs themselves as well as the Jacobian, given the current state vector. Here we'll do that again so that we can wrap it in Cython for use in `scipy.integrate.odeint`.

Start by importing the system of ODEs and the matrix of state variables.

In [ ]:
from scipy2017codegen.chem import load_large_ode

rhs_of_odes, states = load_large_ode()

Now we'll use `sympy.utilities.codegen.codegen` to output C source and header files which can compute the right hand side (RHS) of the ODEs. Here we'll import it and show the documentation:

In [ ]:
from sympy.utilities.codegen import codegen
#codegen?

We just have one expression we're interested in computing, and that is the matrix expression representing the derivatives of our state variables with respect to time: `rhs_of_odes`. What we want `codegen` to do is create a C function that takes in the current values of the state variables and gives us back each of the derivatives.

In [ ]:
[(cf, cs), (hf, hs)] = codegen(('c_odes', rhs_of_odes), language='c')

Note that we've just expanded the outputs into individual variables so we can access the generated code easily. Let's print the source code.

In [ ]:
print(cs)

There are several things here worth noting:

- the state variables are passed in individually
- the state variables in the function signature are out of order
- the output array is passed in as a pointer like in our Fibonacci sequence example, but it has an auto-generated name

Let's address the first issue first. Similarly to what we did in the C printing exercises, let's use a `MatrixSymbol` to represent our state vector instead of a matrix of individual state variables. First, create the `MatrixSymbol` object that is the same shape as our states matrix.

In [ ]:
y = sm.MatrixSymbol('y', *states.shape)

Now we need to replace the use of `y0`, `y1`, etc. in our `rhs_of_odes` matrix with the elements of our new state vector (e.g. `y[0]`, `y[1]`, etc.).

In [ ]:
state_array_map = dict(zip(states, y))
rhs_of_odes_ind = rhs_of_odes.xreplace(state_array_map)
rhs_of_odes_ind[0]

Now let's try to use `codegen` again but this time with `rhs_of_odes` making use of a state vector rather than a container of symbols.

In [ ]:
[(cf, cs), (hf, hs)] = codegen([('c_odes', rhs_of_odes_ind)], language='c')
print(cs)

So by re-writing our expression in terms of a `MatrixSymbol` rather than individual symbols, the function signature of the generated code is cleaned up greatly.

However, we still have the issue of the auto-generated output variable name. To fix this, we can form a matrix *equation* rather than an expression. The name given to the symbol on the left hand side of the equation will then be used for our output variable name.

We'll start by defining a new `MatrixSymbol` that will represent the left hand side of our equation -- the derivatives of each state variable.

In [ ]:
dY = sm.MatrixSymbol('dY', *y.shape)

Now we can form an equation linking the left hand side and the right hand side.

In [ ]:
ode_eq = sm.Eq(dY, rhs_of_odes_ind)

Now we can use `ode_eq` as our expression.

In [ ]:
codegen([('c_odes', ode_eq)], language='c', to_files=True)

Now that we've used `to_files=True`, the `codegen` function writes the `.c` and `.h` files to the filesystem for us. Let's take a look at the result.

In [ ]:
%cat c_odes.c

Now our generated function has a simple and clean signature that will be fairly simple to wrap using Cython.

### Wrapping the Generated Code with Cython

Now we want to wrap the function that was generated `c_odes` with a Cython function. Let's output just the header `c_odes.h` to see just the function signature.

In [ ]:
%cat c_odes.h

So this function signature is what we need to wrap using Cython. Just as with the Fibonacci sequence example, we start by specifying how the extension module will be built.

In [ ]:
render_pyxbld('cy_odes',
              sources=['c_odes.c'],
              include_dirs=[np.get_include()])

**Exercise 2**: Use the template below and the Cython wrapper for the Fibonacci sequence example to generate a function that we can call from Python to evaluate the right hand side of the system of ODEs. Note that we've added a variable `t` to the wrapper function so we can use this function directly with `odeint`, though our C function doesn't make use of it.

```cython
%%cython_pyximport cy_odes
import numpy as np
cimport numpy as np

cdef extern from "c_odes.h":
    # declare the function signature from c_odes.h
    
def cy_odes(np.ndarray[np.float64_t, ndim=1] y, t):
    # use the c_odes function to compute the derivatives
    return dY
```

In [ ]:
%%cython_pyximport cy_odes
import numpy as np
cimport numpy as np

cdef extern from "c_odes.h":
    void c_odes(double *y, double *dY)
    
def cy_odes(np.ndarray[np.float64_t, ndim=1] y, t):
    cdef np.ndarray[np.float64_t, ndim=1] dY = np.empty(y.size, dtype=np.float64)
    c_odes(<double *> y.data, <double *> dY.data)
    return dY

You can test your wrapped implementation with the following cell.

In [ ]:
y_vals = np.random.randn(14)
cy_odes(y_vals, 0)

Now use `odeint` to integrate the equations and plot the results to check that it worked. First we need to import `odeint`.

In [ ]:
from scipy.integrate import odeint

We've also provided a couple convenience functions in our `scipy2017codegen` package that give some reasonable initial conditions for the system and plot the state trajectories, respectively. Start by grabbing some initial values for our state variables and time values.

In [ ]:
from scipy2017codegen.chem import watrad_init, watrad_plot
y_init, t_vals = watrad_init()

Finally we can integrate the equations using our Cython-wrapped C function and plot the results.

In [ ]:
y_vals = odeint(cy_odes, y_init, t_vals)
watrad_plot(t_vals, y_vals)

## 3. Generating and Compiling a C Extension Module Automatically

Here we'll use SymPy's `autowrap` function to automatically generate the C code for an expression, generate a Cython wrapper, compile the wrapper into an extension module, and make the function available as a callable.

In [ ]:
from sympy.utilities.autowrap import autowrap
#autowrap?

In [ ]:
auto_odes = autowrap(ode_eq, backend='cython', tempdir='./autowraptmp')

In [ ]:
type(auto_odes)

`autowrap` essentially does exactly what we just did manually: generate C code to evaluate the right hand side of the system of ODEs, write a Cython wrapper, compile the C code and wrapper into an extension module, and import it for use in Python.

One advantage to wrapping the generated C code manually is that we get fine control over how the function is used from Python. That is, we were able to specify that from the Python side, the input to our wrapper function and its return value are both 1-dimensional `ndarray` objects. We were also able to add in the extra argument `t` for the current time, making the wrapper function fully compatible with `odeint`.

However, `autowrap` just sees that we have a matrix equation where each side is a 2-dimensional array with shape (14, 1). The function returned then expects the input array to be 2-dimensional and it returns a 2-dimensional array.

In [ ]:
y_vals = np.random.randn(14, 1)
auto_odes(y_vals)

This won't work with `odeint`, so we need to write a simple wrapper that massages the input and output and adds an extra argument for `t`.

In [ ]:
def auto_odes_wrapper(y, t):
    dY = auto_odes(y[:, np.newaxis])
    return dY.squeeze()

In [ ]:
auto_odes_wrapper(y_vals.squeeze(), 0)

**Exercise**: As we have seen previously, we can analytically evaluate the Jacobian of our system of ODEs, which can be helpful in numerical integration. Compute the Jacobian of `rhs_of_odes_ind` with respect to `y`, then use `autowrap` to generate a function that evaluates the Jacobian numerically. Finally, write a Python wrapper called `auto_jac_wrapper` to make it compatible with `odeint`.

In [ ]:
jac = rhs_of_odes_ind.jacobian(y)
auto_jac = autowrap(jac, backend='cython', tempdir='./autowraptmp')

def auto_jac_wrapper(y, t):
    return auto_jac(y[:, np.newaxis])

Use the cell below to test. It should print "(14, 14)"

In [ ]:
y_vals = np.random.randn(14)
auto_jac_wrapper(y_vals, 2.0).shape

Finally, we can use our two wrapped functions in `odeint` and compare to our manually-written Cython wrapper result.

In [ ]:
y_vals = odeint(auto_odes_wrapper, y_init, t_vals, Dfun=auto_jac_wrapper)
watrad_plot(t_vals, y_vals)

### Using a Custom Library

In our set of ODEs, there are quite a few instances of $y_i^2$, where $y_i$ is one of the 14 state variables.

In [ ]:
from sympy.utilities.codegen import C99CodeGen
from sympy.printing.ccode import C99CodePrinter

class CustomPrinter(C99CodePrinter):
    def _print_Pow(self, expr):
        return "fastpow({}, {})".format(expr.base, expr.exp)

printer = CustomPrinter()
gen = C99CodeGen(printer=printer)
gen.preprocessor_statements.append('#include "fastpow.h"')

import scipy2017codegen
import os
fastapprox_dir = os.path.join(os.path.dirname(scipy2017codegen.__file__), 'fastapprox')

auto_odes_custom = autowrap(ode_eq, code_gen=gen, backend='cython',
                            tempdir='./autowraptmp2', include_dirs=[fastapprox_dir])

In [ ]:
def auto_odes_custom_wrapper(y, t):
    dY = auto_odes_custom(y[:, np.newaxis])
    return dY.squeeze()

y_vals, info = odeint(auto_odes_custom_wrapper, y_init, t_vals, full_output=True)
watrad_plot(t_vals, y_vals)

In [ ]:
info